In [1]:
import xarray as xr
import numpy as np

ds = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc")

# List of environmental variables of interest
env_vars = ["temperature", "salinity", "u_velocity", "v_velocity"]

ds

<xarray.Dataset> Size: 157GB
Dimensions:          (time: 365, dummySampleDim: 6, zc: 50, xc: 950, yc: 555)
Coordinates:
    time             (time, dummySampleDim) int32 9kB ...
  * xc               (xc) float32 4kB 800.0 1.6e+03 ... 7.592e+05 7.6e+05
  * yc               (yc) float32 2kB 800.0 1.6e+03 ... 4.432e+05 4.44e+05
  * zc               (zc) float32 200B 3.0 6.0 10.0 ... 2.5e+03 2.75e+03 3e+03
Dimensions without coordinates: dummySampleDim
Data variables:
    grid_mapping     int32 4B ...
    LayerDepths      (zc) float32 200B ...
    depth            (yc, xc) float32 2MB ...
    DXxDYy           (yc, xc) float32 2MB ...
    u_velocity       (time, zc, yc, xc) float32 38GB ...
    v_velocity       (time, zc, yc, xc) float32 38GB ...
    elevation        (time, yc, xc) float32 770MB ...
    temperature      (time, zc, yc, xc) float32 38GB ...
    salinity         (time, zc, yc, xc) float32 38GB ...
    ice_thickness    (time, yc, xc) float32 770MB ...
    ice_compactness  (time, yc, xc) float32 770MB ...
    salinity_ice     (time, yc, xc) float32 770MB ...
Attributes: (12/31)
    Conventions:                            CF-1.5
    title:                                  Current data from SINMOD
    institution:                            SINTEF Ocean
    grid_mapping:                           grid_mapping
    grid_mapping_name:                      polar_stereographic
    straight_vertical_longitude_from_pole:  58.0
    ...                                     ...
    CH:                                     0.3
    CI:                                     5.0
    icedyn:                                 F
    tidal_components:                       [0 0 0 0 0 0 0 0 0 0 0]
    history:                                Thu Nov  7 07:52:09 2024: ncrcat ...
    NCO:                                    netCDF Operators version 5.1.3 (H...

In [2]:
import time
import dask
from scipy.spatial.distance import pdist

def process_features(
    file_path,
    variable_name,
    layer_range = (0,1),
    x_range = (0,-1),
    y_range = (0,-1),
    chunks={"time":-1, "zc": -1, "yc": 50, "xc": 50},
    output_path=None
):
    """
    Process layer data for a specified variable in a NetCDF file.
    
    Parameters:
    - file_path (str): Path to the NetCDF file.
    - variable_name (str): Name of the variable to process.
    - layer_range (tuple): Range of layers to process (start, end).
    - x_range (slice, optional): Range of x coordinates to process. If None, process all.
    - output_path (str): Path to save the processed file (optional). If None, the result is not saved.
    
    Returns:
    - xarray.DataArray: The time-averaged bottom layer data.
    """
    time_start = time.time()

    # Open the dataset
    if chunks:
        ds = xr.open_dataset(file_path, chunks=chunks)
    else:
        ds = xr.open_dataset(file_path)

    print(f"\nAccessed the dataset after {time.time() - time_start:.2f} seconds")
    
    # Extract the variable
    if variable_name == "current_speed":
        data_var = (ds["u_velocity"][:, layer_range[0]:layer_range[1], y_range[0]:y_range[1], x_range[0]:x_range[1]]**2 + \
                   ds["v_velocity"][:, layer_range[0]:layer_range[1], y_range[0]:y_range[1], x_range[0]:x_range[1]]**2)**0.5
    else:
        data_var = ds[variable_name][:, layer_range[0]:layer_range[1], y_range[0]:y_range[1], x_range[0]:x_range[1]]

    # Calculate pairwise distances between points
    lon, lat = np.meshgrid(data_var["xc"].values, data_var["yc"].values)

    coordinates = np.vstack([lon.ravel(), lat.ravel()]).T

    try:
        pairwise_distances = pdist(coordinates, metric='euclidean')
    except MemoryError:
        print("Memory Error: Too many points to calculate pairwise distances")
        pairwise_distances = None
    ds.close()

    print(f"\nExtracted the layer data after {time.time() - time_start:.2f} seconds.\n\nStarting computation of statistics...")

    # Step 4: Calculate statistics across time
    # TODO: Decide on desired features
    time_avg_bottom_layer = data_var.mean(dim="time", skipna=True)
    time_percentiles = data_var.quantile([0.1,0.9], dim="time", skipna=True)

    print(f"\nComputed statistics after {time.time() - time_start:.2f} seconds")

    # Create a new DataArray with the (mean, 10th, 90th) percentiles and explicitly define the 'stat' dimension
    # Concatenate mean and percentiles in one line, drop 'quantile' and concatenate all together
    stats_array = xr.concat([time_avg_bottom_layer, time_percentiles.sel(quantile=0.1).drop_vars("quantile"), time_percentiles.sel(quantile=0.9).drop_vars("quantile")], dim="stat").rename(f"{variable_name}_features")

    # Name each value of the first dimension
    stats_array = stats_array.assign_coords(stat=["mean", "10th_percentile", "90th_percentile"])

    # Save to output file if specified
    if output_path:
        stats_array.to_netcdf(output_path)
    
    return stats_array, pairwise_distances

In [ ]:
temp_test, pairwise_dist = process_features("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "temperature", 
                                             x_range=(65,75), y_range=(0,10), chunks=None)

# Save as a NetCDF file
temp_array.to_netcdf("../1_data/4_interim/temperature_features_10x10_test.nc", mode="w")


Accessed the dataset after 0.01 seconds

Extracted the layer data after 0.01 seconds.

Starting computation of statistics...


/cluster/home/haroldh/.conda/envs/spGDMM/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



Computed statistics after 6.24 seconds


In [ ]:
# Create array to be transformed to biological space

temp_test = xr.open_dataset("../1_data/4_interim/temperature_features_10x10_test.nc")
salinity_test = xr.open_dataset("../1_data/4_interim/salinity_features_10x10_test.nc")

SINMOD_features_test = xr.Dataset({
    'temperature': temp_test['temperature_features'],
    'salinity': salinity_test['salinity_features']
})

# SEND TO R FOR I SPLINE 
SINMOD_features_test.to_netcdf("/cluster/home/haroldh/spGDMM/1_data/4_interim/SINMOD_features_10x10_test.nc", mode='w')

<xarray.Dataset> Size: 1kB
Dimensions:               (xc: 10, yc: 10, zc: 1, stat: 3)
Coordinates:
  * xc                    (xc) float32 40B 5.28e+04 5.36e+04 ... 5.92e+04 6e+04
  * yc                    (yc) float32 40B 800.0 1.6e+03 ... 7.2e+03 8e+03
  * zc                    (zc) float32 4B 3.0
  * stat                  (stat) <U15 180B 'mean' ... '90th_percentile'
Data variables:
    temperature_features  (stat, zc, yc, xc) float32 1kB ...

In [25]:
salinity_array, _ = process_features("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "salinity",
                                     chunks=None)


Accessed the dataset after 0.01 seconds
Memory Error: Too many points to calculate pairwise distances

Extracted the layer data after 0.01 seconds.

Starting computation of statistics...


/cluster/home/haroldh/.conda/envs/spGDMM/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



Computed statistics after 91.46 seconds


In [24]:
temp_array, _ = process_features("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "temperature", chunks=None)
temp_array


Accessed the dataset after 0.01 seconds
Memory Error: Too many points to calculate pairwise distances

Extracted the layer data after 0.01 seconds.

Starting computation of statistics...


/cluster/home/haroldh/.conda/envs/spGDMM/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,



Computed statistics after 95.28 seconds


<xarray.DataArray 'temperature_features' (stat: 3, zc: 1, yc: 554, xc: 949)> Size: 6MB
array([[[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [ 8.95036  ,  8.95442  ,  8.950777 , ...,  8.549332 ,
           8.547776 ,  8.547671 ],
         [ 8.925162 ,  8.929397 ,  8.924648 , ...,  8.54698  ,
           8.545511 ,  8.545669 ],
         [ 8.9004345,  8.902989 ,  8.898687 , ...,  8.544727 ,
           8.543233 ,  8.543532 ]]],


       [[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
...
           6.794153 ,  6.79574  ],
         [ 7.172009 ,  7.1859736,  7.1726437, ...,  6.794629 ,
           6.798914 ,  6.7966924],
         [ 7.078536 ,  7.088216 ,  7.0767903, ...,  6.797803 ,
           6.791138 ,  6.796375 ]]],


       [[[       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         [       nan,        nan,        nan, ...,        nan,
                 nan,        nan],
         ...,
         [11.117356 , 11.117356 , 11.11688  , ..., 10.9059725,
          10.901688 , 10.900101 ],
         [11.131639 , 11.131639 , 11.131798 , ..., 10.9070835,
          10.903275 , 10.901529 ],
         [11.120688 , 11.122593 , 11.122593 , ..., 10.905497 ,
          10.904861 , 10.903116 ]]]], dtype=float32)
Coordinates:
  * xc       (xc) float32 4kB 800.0 1.6e+03 2.4e+03 ... 7.584e+05 7.592e+05
  * yc       (yc) float32 2kB 800.0 1.6e+03 2.4e+03 ... 4.424e+05 4.432e+05
  * zc       (zc) float32 4B 3.0
  * stat     (stat) <U15 180B 'mean' '10th_percentile' '90th_percentile'

In [51]:
SINMOD_features = xr.Dataset({
    'temperature': temp_array,
    'salinity': salinity_array
})

#SINMOD_features.sel(stat="mean", xc=574, yc=122)
SINMOD_features = SINMOD_features.squeeze('zc', drop=True)


SINMOD_features['stat'].values

array(['mean', '10th_percentile', '90th_percentile'], dtype='<U15')

In [59]:
# Get training data at sample locations 
sampled_locations = pd.read_csv("/cluster/home/haroldh/spGDMM/1_data/1_raw/synthetic_abundance/sampled_locations.csv")

# Initialize an empty list to store the results
results = np.empty((len(sampled_locations), len(SINMOD_features.data_vars) * len(SINMOD_features['stat'].values)))

print(results.shape)

for i, row in sampled_locations.iterrows():
    x = row['x']
    y = row['y']
    j=0
    for var in SINMOD_features.data_vars:
        for stat in SINMOD_features['stat'].values:

            # Use the values for selection
            results[i,j] = SINMOD_features[var].isel(xc=x, yc=y).sel(stat=stat).values
            
            j+=1
            
            # Save results to a CSV file

# Generate column names
column_names = [f"{var}_{stat}" for var in SINMOD_features.data_vars for stat in SINMOD_features['stat'].values]

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=column_names)

# Save results to a CSV file
results_df.to_csv("/cluster/home/haroldh/spGDMM/1_data/2_processed/training/midnor_training.csv", index=False)


(10, 6)


In [60]:
import pandas as pd
# Get from R and then transform to biological space
X_GDM_predictors_bs = pd.read_csv("/cluster/home/haroldh/spGDMM/1_data/2_processed/prediction/X_GDM_predictors_bs.csv")

# Get beta values
beta_posts = pd.read_csv("/cluster/home/haroldh/spGDMM/4_trained_models/mod1_SINMOD_post_samples.csv")

In [65]:
beta_posts

,Unnamed: 0,beta[1],beta[2],beta[3],beta[4],beta[5],beta[6],beta[7],beta[8],beta[9],...,log_beta[20],log_beta[21],log_beta[22],log_beta[23],log_beta[24],log_beta[25],log_beta[26],log_beta[27],log_beta[28],sigma2
0,1,0.000478,9.930778e-03,0.003656,7.125718e-08,0.014109,1.106291e-04,1.125708e-02,7.636529e-08,0.000028,...,-15.495421,-4.227061,-10.672350,-14.278251,-18.263550,-5.757410,-11.559771,-9.927411,-18.952724,0.069987
1,2,0.003335,2.662246e-04,0.003304,1.628768e-04,0.000152,2.985620e-02,8.528794e-04,7.348121e-07,0.050950,...,-15.852645,-12.693305,-6.584980,-16.647864,-11.819031,-2.646018,-19.846806,-2.330011,-13.586546,0.088419
2,3,0.023321,2.089346e-06,0.000186,8.936939e-06,0.000972,1.783313e-04,5.630209e-03,2.088386e-05,0.000021,...,-18.080706,-5.269440,-1.905990,-18.660712,-12.285211,-6.893094,-24.665143,-8.145978,-2.487447,0.073343
3,4,0.000144,2.808116e-08,0.009319,8.291733e-03,0.009663,1.586285e-02,2.658822e-07,4.945788e-07,0.002239,...,-17.209824,-17.147836,-4.489584,-6.990162,-11.034089,-7.468274,-27.396524,-12.249758,-2.207818,0.086931
4,5,0.001792,2.279459e-08,0.009091,1.353431e-02,0.002139,1.923023e-02,3.681542e-04,5.237850e-05,0.006350,...,-15.496724,-15.897066,-5.630353,-8.626385,-11.049829,-16.633490,-25.304481,-11.130121,-6.622800,0.080367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,0.004968,8.251882e-06,0.000607,8.097699e-08,0.000073,1.512722e-06,3.136766e-11,2.037171e-07,0.000093,...,-8.070512,-12.340637,-5.056859,-8.259745,-6.501163,-20.745235,-1.937295,-16.956606,-2.484583,0.088292
9996,9997,0.000696,4.351274e-06,0.000016,3.084258e-05,0.008323,6.609359e-08,1.870501e-08,8.478245e-05,0.000529,...,-14.607733,-16.660583,-11.090532,-3.871266,-8.154873,-11.621417,-2.959614,-10.514247,-7.118994,0.080290
9997,9998,0.003540,5.992698e-05,0.000003,6.965353e-06,0.000129,2.347525e-07,3.049421e-08,1.384463e-01,0.015758,...,-13.585902,-21.006178,-7.815743,-4.016108,-2.444322,-5.406345,-4.042643,-8.914990,-1.775172,0.082622
9998,9999,0.000004,5.836501e-02,0.000100,7.393795e-09,0.001991,2.907282e-05,7.328415e-08,8.885350e-03,0.038321,...,-27.718098,-14.380131,-9.989908,-19.643234,-8.936693,-9.736173,-2.449462,-13.087522,-3.813877,0.080732
